pip install opencv-python

In [2]:
import cv2

In [5]:
image1 = cv2.imread(r'C:\Users\sarthak.agarwal\Pictures\penguins.jpg')
image1

array([[[201, 163, 109],
        [201, 163, 109],
        [201, 163, 109],
        ...,
        [173, 136,  86],
        [173, 135,  87],
        [174, 136,  88]],

       [[201, 163, 109],
        [201, 163, 109],
        [201, 163, 109],
        ...,
        [173, 136,  86],
        [173, 135,  87],
        [173, 135,  87]],

       [[201, 163, 109],
        [201, 163, 109],
        [201, 163, 109],
        ...,
        [173, 136,  86],
        [173, 135,  87],
        [173, 135,  87]],

       ...,

       [[184, 173, 165],
        [184, 173, 165],
        [184, 173, 165],
        ...,
        [150, 134, 128],
        [152, 136, 130],
        [153, 137, 131]],

       [[184, 173, 165],
        [184, 173, 165],
        [184, 173, 165],
        ...,
        [150, 134, 128],
        [151, 135, 129],
        [152, 136, 130]],

       [[186, 173, 165],
        [186, 173, 165],
        [186, 173, 165],
        ...,
        [150, 134, 128],
        [151, 135, 129],
        [152, 136, 130]]

In [4]:
cv2.imshow("Penguins",image1)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [20]:
import numpy as np, cv2

cap = cv2.VideoCapture(0)

image1 = cv2.imread(r'C:\Users\sarthak.agarwal\Pictures\penguins.jpg')

# For face detection
face_cascade = cv2.CascadeClassifier("./haarcascade_frontalface_alt.xml")
skip = 0
face_data = []
face_section = image1
dataset_path = './data/'

filename = input("Enter the name of the person")

while True:
    ret, frame = cap.read()
 
    if ret == False:
        continue
    grayFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(frame, 1.3, 5)
    faces = sorted(faces,key=lambda f:f[2]*f[3])
   
    for face in faces[-1:]:
        x,y,w,h = face
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)

        offset = 10
        face_section = frame[y-offset : y+h+offset, x-offset : x+w+offset]
        #print(type(face_section), face_section.shape)
        face_section = np.array(cv2.resize(face_section,(100,100)))
        skip += 1
        if(skip%10 == 0):
            face_data.append(face_section)
            print(len(face_data))


    cv2.imshow("Capturing WebCam content",frame)
    cv2.imshow("Face Section", face_section)
    
    key_pressed = cv2.waitKey(1) & 0xFF
    if(key_pressed) == ord('q'):
        break

face_data = np.asarray(face_data)
face_data = face_data.reshape((face_data.shape[0], -1))
print(face_data.shape)

np.save(dataset_path + filename + '.npy', face_data)
print('Data successfully saved to '+ dataset_path + filename + '.npy')

cap.release()
cv2.destroyAllWindows()

Enter the name of the personavinash
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
(49, 30000)
Data successfully saved to ./data/avinash.npy


In [1]:
import os,cv2, numpy as np, pickle

from sklearn.neighbors import KNeighborsClassifier

face_cascade = cv2.CascadeClassifier("./haarcascade_frontalface_alt.xml")
skip = 0
face_data = []
face_section = []
dataset_path = './data/'
trainData = []
labels = []
classId = 1
names = {}


for file in os.listdir(dataset_path):
    if file.endswith('.npy'):
        data_item = np.load(dataset_path+ file)
        trainData.append(data_item)
#         print(data_item.shape)
        target = classId * np.ones((data_item.shape[0]),)
        names[classId] = file[:-4]
        labels.append(target)
        classId += 1

faceTrainData = np.concatenate(trainData, axis = 0)
faceLabelData = np.concatenate(labels, axis = 0).reshape((-1,1))

# print(len(trainData))
print(faceTrainData.shape)
print(faceLabelData.shape)

knnClassifier = KNeighborsClassifier()
knnClassifier.fit(faceTrainData,faceLabelData)
print(knnClassifier.score(faceTrainData,faceLabelData))

pickle.dump(knnClassifier,open('model.sav','wb'))

(249, 30000)
(249, 1)


C:\Users\sarthak.agarwal\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:34: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


0.9919678714859438


In [3]:
import os,cv2, numpy as np, pickle

knnClassifier = pickle.load(open('model.sav','rb'))
face_cascade = cv2.CascadeClassifier("./haarcascade_frontalface_alt.xml")

cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
 
    if ret == False:
        print("Error")
        continue
    faces = face_cascade.detectMultiScale(frame, 1.3, 5)
   
    for face in faces:
        x,y,w,h = face
        

        offset = 10
        face_section = frame[y-offset : y+h+offset, x-offset : x+w+offset]
        face_section = np.array(cv2.resize(face_section,(100,100)))
#         print(face_section.shape, face_section.flatten().reshape(1,-1).shape)
#         a = input("Hey")
        predictedClass = knnClassifier.predict(face_section.flatten().reshape(1,-1))
#         print(predictedClass)
        cv2.putText(frame,names[int(predictedClass)],(x,y-10),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        #print(type(face_section), face_section.shape)
#         face_section = np.array(cv2.resize(face_section,(100,100)))
#         skip += 1
#         if(skip%10 == 0):
#             face_data.append(face_section)
#             print(len(face_data))


    cv2.imshow("Capturing WebCam content",frame)
#     cv2.imshow("Face Section", face_section)
    
    key_pressed = cv2.waitKey(1) & 0xFF
    if(key_pressed) == ord('q'):
        print("Here")
        break

cap.release()
cv2.destroyAllWindows()

Here


In [26]:
import os,cv2, numpy as np, pickle
from shutil import move

locationToReadFrom = 'C:/Users/sarthak.agarwal/Pictures/Camera Roll/'

face_cascade = cv2.CascadeClassifier("./haarcascade_frontalface_alt.xml")
for file in os.listdir(locationToReadFrom):
    if(file.endswith('.jpg')):
        print(file)
        imageInput = cv2.imread(locationToReadFrom + file)
        
        print(imageInput.shape)

        face = face_cascade.detectMultiScale(imageInput, 1.3, 5)
        print(face)
        if(len(face) > 0):
            (x,y,w,h) = face[0]

            offset = 10
            face_section = imageInput[y-offset : y+h+offset, x-offset : x+w+offset]
            face_section = np.array(cv2.resize(face_section,(100,100)))
            print(face_section.shape, face_section.flatten().reshape(1,-1).shape)
            #         a = input("Hey")
            predictedClass = knnClassifier.predict(face_section.flatten().reshape(1,-1))
            print(predictedClass)
            print(names[int(predictedClass)])
            destPath = locationToReadFrom + names[int(predictedClass)] 
            if not os.path.exists(destPath):
                os.mkdir(destPath)
                os.chmod(destPath, 0o777)
                os.system('chmod 777 -R *')
                
            move(locationToReadFrom + file, destPath+ "/" + file)
            

WIN_20190725_19_15_14_Pro.jpg
(720, 1280, 3)
[[605 250 276 276]]
(100, 100, 3) (1, 30000)
[4.]
sarthak
WIN_20190725_19_15_18_Pro.jpg
(720, 1280, 3)
[[666 238 311 311]]
(100, 100, 3) (1, 30000)
[4.]
sarthak
WIN_20190725_19_15_19_Pro.jpg
(720, 1280, 3)
()
WIN_20190725_19_32_18_Pro.jpg
(720, 1280, 3)
[[768 194 177 177]]
(100, 100, 3) (1, 30000)
[4.]
sarthak
WIN_20190725_19_32_25_Pro.jpg
(720, 1280, 3)
()
WIN_20190725_19_38_10_Pro.jpg
(720, 1280, 3)
[[547 252 356 356]]
(100, 100, 3) (1, 30000)
[1.]
amit
